# Projeto Assistente Virtual da Receita Federal

## Preparando o ambiente

In [ ]:
# ============= setup =============
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from dotenv import load_dotenv
import nest_asyncio

from IPython.display import display, HTML
from llama_index.core.prompts import PromptTemplate

from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.gemini import Gemini

from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core.agent import ReActAgent
from workflows.events import StopEvent

import warnings
warnings.filterwarnings('ignore')

In [27]:
# Aplicar o nest_asyncio - a biblioteca asyncio controla a execução de funções assíncronas
nest_asyncio.apply()

# Carregar as variáveis de ambiente
load_dotenv(override=True)

# acesse o site da openai e crie uma chave de api
groq_api_key = os.getenv('GROQ_API_KEY') # modelo llm - https://groq.com/
hf_api_key = os.getenv('HF_API_KEY') # modelo de embedding - https://huggingface.co/
gemini_api_key = os.getenv('GOOGLE_API_KEY')

In [ ]:
# Configuração global  
# llm
Settings.llm = Gemini(
    model="models/gemini-2.5-flash", 
    api_key=gemini_api_key
)  

# modelo de embedding 
Settings.embed_model = GeminiEmbedding(
    model_name="models/embedding-001",
    token=hf_api_key
)

Settings.chunk_size = 1024 # antes 512
Settings.chunk_overlap = 200 # antes 64

In [52]:
# ============= carregar os dados =============
from llama_index.core import SimpleDirectoryReader

pf_docs = SimpleDirectoryReader(input_files=["./knowledge_base/faq_iprrf_pf.pdf"]).load_data()      # manual de perguntas e respostas para PF
pj_docs = SimpleDirectoryReader(input_files=["./knowledge_base/faq_pj.pdf"]).load_data()            # manual de perguntas e respostas para PJ
pj_mei_docs = SimpleDirectoryReader(input_files=["./knowledge_base/faq_pj_mei.pdf"]).load_data()    # manual de perguntas e respostas para PJ MEI

In [53]:
# ============= PF =============
# template de prompt
template_prompt_acessivel = (
    "As informações de contexto relevantes estão abaixo.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Sua tarefa é atuar como um assistente de imposto de renda amigável. "
    "Use apenas as informações de contexto fornecidas para responder à pergunta do usuário. "
    "Responda em português, de forma simples, direta e fácil de entender, como se estivesse explicando para alguém sem conhecimento do assunto. "
    "Evite jargão técnico e linguagem complicada.\n\n"
    "Pergunta do usuário: {query_str}\n"
    "Resposta Acessível: "
)

prompt_acessivel = PromptTemplate(template_prompt_acessivel)


### Recuperação de reposta relacionado a pessoa física

In [33]:
# indexação
pf_index = VectorStoreIndex.from_documents(pf_docs)

# criando o motor de busca
pf_query_engine = pf_index.as_query_engine(
    similarity_top_k=3,                         # 
    text_qa_template=prompt_acessivel           # 
)

In [34]:
# Fazendo a pergunta de teste
response = pf_query_engine.query("Como declaro o aluguel que recebi de um imóvel?")

### Exemplo para comparação (modelo llama3.3-8b)

In [25]:
# Exibindo a resposta
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

print("\n--- Fontes Consultadas ---")

# Verifique se existem nós de origem antes de tentar iterar sobre eles
if response.source_nodes:
    for i, node in enumerate(response.source_nodes):
        # Extrai as informações do metadado do nó
        file_name = node.metadata.get('file_name', 'N/A')
        page_label = node.metadata.get('page_label', 'N/A')
        
        # Extrai o score de similaridade
        similarity_score = node.score
        
        # Extrai o conteúdo do nó (o texto)
        node_content = node.get_content()

        print(f"Fonte {i+1}:")
        print(f"  - Documento: {file_name}")
        print(f"  - Página: {page_label}")
        print(f"  - Relevância (Score): {similarity_score:.4f}")
        # Exibe um trecho do texto para contexto
        print(f"  - Trecho do Texto: \"{node_content[:200].strip()}...\"\n")
else:
    print("Nenhuma fonte foi utilizada para gerar esta resposta.")


--- Fontes Consultadas ---
Fonte 1:
  - Documento: faq_iprrf_pf.pdf
  - Página: 37
  - Relevância (Score): 0.5692
  - Trecho do Texto: "37 
RETIFICAÇÃO - PRAZO 
044 — Há limite de prazo para a retificação da declaração? 
Sim. Extingue-se em cinco anos o direito de o contribuinte retificar a declaração de rendimentos, inclusive 
quanto..."

Fonte 2:
  - Documento: faq_iprrf_pf.pdf
  - Página: 106
  - Relevância (Score): 0.5659
  - Trecho do Texto: "106 
RENDIMENTOS DE IMÓVEL CEDIDO 
210 — Como tratar os rendimentos produzidos por imóvel cujo direito de exploração tenha sido 
cedido, por meio de contrato, a terceiros? 
Esses rendimentos são tribu..."

Fonte 3:
  - Documento: faq_iprrf_pf.pdf
  - Página: 104
  - Relevância (Score): 0.5534
  - Trecho do Texto: "0190”) da DAA. 
Atenção:  
Para determinação da base de cálculo sujeita ao recolhimento mensal obrigatório (carnê-leão), no 
caso de rendimentos de aluguéis de imóveis pagos por pessoa física, e no ca..."



In [35]:
# Exibindo a resposta
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

print("\n--- Fontes Consultadas ---")

# Verifique se existem nós de origem antes de tentar iterar sobre eles
if response.source_nodes:
    for i, node in enumerate(response.source_nodes):
        # Extrai as informações do metadado do nó
        file_name = node.metadata.get('file_name', 'N/A')
        page_label = node.metadata.get('page_label', 'N/A')
        
        # Extrai o score de similaridade
        similarity_score = node.score
        
        # Extrai o conteúdo do nó (o texto)
        node_content = node.get_content()

        print(f"Fonte {i+1}:")
        print(f"  - Documento: {file_name}")
        print(f"  - Página: {page_label}")
        print(f"  - Relevância (Score): {similarity_score:.4f}")
        # Exibe um trecho do texto para contexto
        print(f"  - Trecho do Texto: \"{node_content[:200].strip()}...\"\n")
else:
    print("Nenhuma fonte foi utilizada para gerar esta resposta.")


--- Fontes Consultadas ---
Fonte 1:
  - Documento: faq_iprrf_pf.pdf
  - Página: 184
  - Relevância (Score): 0.7833
  - Trecho do Texto: "Sim, desde que o encargo tenha sido do locador.  
Podem ser excluídos do valor do aluguel recebido, quando o encargo tenha sido do locador, as quantias 
relativas ao pagamento do Imposto sobre a Propr..."

Fonte 2:
  - Documento: faq_iprrf_pf.pdf
  - Página: 105
  - Relevância (Score): 0.7578
  - Trecho do Texto: "Os rendimentos do 
aluguel são tributáveis em nome do filho. 
Se não houver escritura averbada, o pai, ao relacionar o imóvel em sua Declaração de Bens e Direitos, informa 
que os rendimentos respecti..."

Fonte 3:
  - Documento: faq_iprrf_pf.pdf
  - Página: 105
  - Relevância (Score): 0.7564
  - Trecho do Texto: "(Decreto-Lei nº 5.844, de 23 de setembro de 1943, art. 100, parágrafo único, alínea ‘a’; Regulamento 
do Imposto sobre a Renda – RIR/2018, arts. 744, 763 e 781, aprovado pelo Decreto nº 9.580, de 
22..."



### Pessoa jurídica

In [54]:
# indexação
pj_index = VectorStoreIndex.from_documents(pj_docs)

# criando o motor de busca
pj_query_engine = pj_index.as_query_engine(
    similarity_top_k=3,                         # 
    text_qa_template=prompt_acessivel           # 
)

In [ ]:
response = pj_query_engine.query(
    "Em que casos a Contribuição para o PIS/Pasep e a Cofins são apuradas pelo regime de caixa?"
    )

In [56]:
# Exibindo a resposta
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

print("\n--- Fontes Consultadas ---")

# Verifique se existem nós de origem antes de tentar iterar sobre eles
if response.source_nodes:
    for i, node in enumerate(response.source_nodes):
        # Extrai as informações do metadado do nó
        file_name = node.metadata.get('file_name', 'N/A')
        page_label = node.metadata.get('page_label', 'N/A')
        
        # Extrai o score de similaridade
        similarity_score = node.score
        
        # Extrai o conteúdo do nó (o texto)
        node_content = node.get_content()

        print(f"Fonte {i+1}:")
        print(f"  - Documento: {file_name}")
        print(f"  - Página: {page_label}")
        print(f"  - Relevância (Score): {similarity_score:.4f}")
        # Exibe um trecho do texto para contexto
        print(f"  - Trecho do Texto: \"{node_content[:200].strip()}...\"\n")
else:
    print("Nenhuma fonte foi utilizada para gerar esta resposta.")


--- Fontes Consultadas ---
Fonte 1:
  - Documento: faq_pj.pdf
  - Página: 648
  - Relevância (Score): 0.8507
  - Trecho do Texto: "648 
 
Veja ainda: Contribuintes da Contribuição para o 
PIS/Pasep e da Cofins, incidentes sobre a 
receita ou o faturamento: 
 Pergunta 001 
 Contribuintes da Contribuição para o 
PIS/Pasep e da Cofi..."

Fonte 2:
  - Documento: faq_pj.pdf
  - Página: 641
  - Relevância (Score): 0.8451
  - Trecho do Texto: "641 
 
Notas: 
1) Na receita bruta não se incluem os tributos não 
cumulativos cobrados, destacadamente, do 
comprador ou contratante pelo vendedor dos 
bens ou pelo prestador dos serviços na 
condiçã..."

Fonte 3:
  - Documento: faq_pj.pdf
  - Página: 642
  - Relevância (Score): 0.8356
  - Trecho do Texto: "642 
 
Notas: 
No caso de contrato de concessão de serviços 
públicos, a receita decorrente da construção, 
recuperação, reforma, ampliação ou 
melhoramento da infraestrutura, cuja 
contrapartida seja..."



# Criando o Agente

In [ ]:

# Definição das ferramentas
query_engine_tools = [
    QueryEngineTool(
        query_engine=pf_query_engine,
        metadata=ToolMetadata(
            name="imposto_renda_pessoa_fisica",
            description=(
                "Útil para responder perguntas sobre a declaração do Imposto de Renda "
                "para Pessoas Físicas (IRPF). Cobre tópicos como deduções, isenções, "
                "como declarar bens, rendimentos de aluguel e ganho de capital."
            ),
        ),
    ),
    
    QueryEngineTool(
        query_engine=pj_query_engine,
        metadata=ToolMetadata(
            name="imposto_renda_pessoa_juridica",
            description=(
                "Fornece respostas sobre a tributação de empresas (Pessoas Jurídicas). "
                "Ideal para dúvidas sobre impostos como IRPJ, CSLL, PIS/Pasep e Cofins, "
                "e regimes de apuração como Lucro Real e Lucro Presumido."
            ),
        ),
    ),
]

# LLM
llm = Gemini(model="gemini-1.5-flash", api_key=gemini_api_key)

# Agente
agent = ReActAgent(
    tools=query_engine_tools,
    llm=llm,
    verbose=True
)

In [ ]:
response = agent.run("Como declaro o aluguel que recebi de um imóvel?")

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent


In [ ]:
display(HTML(f'<p style="font-size:20px">{response}</p>'))

In [ ]:
response = agent.run("Em que casos a Contribuição para o PIS/Pasep e a Cofins são apuradas pelo regime de caixa?")

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent


In [ ]:
display(HTML(f'<p style="font-size:20px">{response}</p>'))